## initiate

In [0]:
%run ./common_module

In [0]:
fillna_value = dbutils.widgets.get("fillna_value")
print(fillna_value)

## read table

In [0]:
pred_base_table = dbutils.widgets.get("pred_base_table")
print(pred_base_table)

In [0]:
pred_base = spark.table(pred_base_table)
display(pred_base)

## drop useless rows

In [0]:
pred_base = pred_base.select('*').where(col("PO_PP_IND") == 'PO')
pred_base = pred_base.select('*').where(col("SUBSCR_TYPE") != 'D')


## drop useless column

In [0]:
pred_base = pred_base.drop("CURR_IMEI_BAND_FREQ","IMEI_TAC_PRICE","HS_RELEASE_TENURE","HS_MKT_RELEASE_TENURE","CURR_IMEI_TYPE" # imei-related are all overdated DATA
                       ,"P6M_MIN_NET_INV_AMT" # DUP MEANING
                       ,"LAST_CHANNEL_TYPE" # WRONG DATA
                       , "PO_PP_IND") # NO-NEED DATA


In [0]:
display(pred_base)

## transformation - correct data type

In [0]:
pred_base = pred_base.withColumn("AGE_BAND",col("AGE_BAND").cast("string"))
pred_base = pred_base.withColumn("CURR_HS_USE_TIME",col("CURR_HS_USE_TIME").cast("int"))
pred_base = pred_base.withColumn("AGE_BL35_IND",col("AGE_BL35_IND").cast("string"))
pred_base = pred_base.drop("Y_GA_FLAG")

## categorical data: transformation - fix weird data

In [0]:
pred_base = pred_base.withColumn("CITY_NAME",when(col('CITY_NAME') == '臺中市    ', 'Taichung')\
.when(col('CITY_NAME') == '臺北市    ', 'Taipei')\
.when(col('CITY_NAME') == 'null', 'null')\
.when(col('CITY_NAME') == '臺東縣    ', 'Taitung')\
.when(col('CITY_NAME') == '金門縣    ', 'Kinmen')\
.when(col('CITY_NAME') == '高雄市    ', 'Kaoshiung')\
.when(col('CITY_NAME') == '澎湖縣    ', 'Penghu')\
.when(col('CITY_NAME') == '南投縣    ', 'Nantou')\
.when(col('CITY_NAME') == '新竹市    ', 'HsinchuCity')\
.when(col('CITY_NAME') == '屏東縣    ', 'Pingtong')\
.when(col('CITY_NAME') == '新竹縣    ', 'HsinchuCounty')\
.when(col('CITY_NAME') == '嘉義市    ', 'ChiayiCity')\
.when(col('CITY_NAME') == '基隆市    ', 'Keelung')\
.when(col('CITY_NAME') == '嘉義縣    ', 'ChiayiCounty')\
.when(col('CITY_NAME') == '雲林縣    ', 'Yunling')\
.when(col('CITY_NAME') == '宜蘭縣    ', 'Yilan')\
.when(col('CITY_NAME') == '臺南市    ', 'Tainan')\
.when(col('CITY_NAME') == '花蓮縣    ', 'Hualien')\
.when(col('CITY_NAME') == '苗栗縣    ', 'Miaoli')\
.when(col('CITY_NAME') == '連江縣    ', 'Lienchaing')\
.when(col('CITY_NAME') == '新北市    ', 'NewTaipei')\
.when(col('CITY_NAME') == '桃園市    ', 'Taoyuan')\
.when(col('CITY_NAME') == '彰化縣    ', 'Changhua')\

)

In [0]:
pred_base = pred_base.withColumn("SMS_PROM_IND",when(col('SMS_PROM_IND') == 'null', 'null')\
.when(col('SMS_PROM_IND') == 'Y', 'Y')\
.when(col('SMS_PROM_IND') == 'R', 'N')\
.when(col('SMS_PROM_IND') == 'N', 'N')\
.when(col('SMS_PROM_IND') == '1', 'Y')\
)

## numeric data: transformation - see statistical summary in numeric data

In [0]:
fillna_value = dbutils.widgets.get("fillna_value")
print(fillna_value)

In [0]:
pred_base = fillna_for_numeric_data(pred_base,['ID',],int(fillna_value))
display(pred_base)

## get_dummy

In [0]:
pred_base = pred_base.drop('DATA_MONTH')

In [0]:
pred_base = get_dummy(pred_base,['ID'])
display(pred_base)

## drop useless dummy column

In [0]:
pred_base = pred_base.drop('SUBSCR_TYPE_v_', 'SMS_PROM_IND_n')

## write to DB

In [0]:
#write to database
pred_base.write.format('delta').saveAsTable('DB.pred_dummy',mode = 'overwrite')